In [1]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K

### Konfigurasi Parameter

In [2]:
img_width, img_height = 224, 224

train_data_dir = 'dataset/train/'
# valid_data_dir = 'dataset/valid/'

epochs = 100
batch_size = 4

### LOAD DATASET

In [3]:
# Mendapatkan Banyaknya Jumlah Class

dirs_train = os.listdir(train_data_dir)
# dirs_valid = os.listdir(valid_data_dir)

# if len(dirs_train) != len(dirs_valid):
#     print('Jumlah Class tidak sama !')
#     classes_number = -1
# else:
#     classes_number = len(dirs_train)
classes_number = len(dirs_train)

In [5]:
classes_number
print(dirs_train)

['allan', 'faiz', 'ilyas', 'indah', 'mitha', 'naufal', 'sando', 'syifa', 'tasya', 'victor']


In [6]:
# Menghitung Jumlah Gambar pada Dataset Train

nb_train_sample = 0

for clz in dirs_train:
    path_class = train_data_dir + "/" + clz

    n_img = len(os.listdir(path_class))

    nb_train_sample += n_img

train_batch = nb_train_sample / batch_size

In [8]:
# # Menghitung Jumlah Gambar pada Dataset Valid

# nb_valid_sample = 0

# for clz in dirs_valid:
#     path_class = valid_data_dir + "/" + clz

#     n_img = len(os.listdir(path_class))

#     nb_valid_sample += n_img

# valid_batch = nb_valid_sample / batch_size

### Buat Arsitektur CNN

In [7]:
def create_model(input_shape):
    # Initial Aristektur
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4, 4), input_shape=input_shape),
        Activation('relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same"),
        Activation('relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same"),
        Activation('relu'),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same"),
        Activation('relu'),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same"),
        Activation('relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', 
                optimizer='adam', metrics=['accuracy'])
    
    return model

### Mengetahui Struktur Image

(C, W, H) --> Channel First <br>
(W, H, C) --> Channel Last

In [12]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [9]:
input_shape

(224, 224, 3)

### Preprocessing Gambar memakai ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(
    rescale= 1. / 255.,
    horizontal_flip= True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# valid_datagen = ImageDataGenerator(
#     rescale= 1. / 255.
# )

# valid_generator = valid_datagen.flow_from_directory(
#     valid_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical'
# )

Found 80 images belonging to 10 classes.


### TRAINING

In [14]:
model = create_model(input_shape)

model.fit(
    train_generator,
    steps_per_epoch=train_batch,
    epochs=epochs,
    # validation_steps=valid_batch,
    # validation_data=valid_generator,
    verbose=1,
    # workers=8,
)

Epoch 1/100
20/20 [==============================] - 23s 1s/step - loss: 34.0765 - accuracy: 0.0875
Epoch 2/100
20/20 [==============================] - 23s 1s/step - loss: 17.6676 - accuracy: 0.1625
Epoch 3/100
20/20 [==============================] - 21s 1s/step - loss: 10.4798 - accuracy: 0.3000
Epoch 4/100
20/20 [==============================] - 21s 1s/step - loss: 8.3054 - accuracy: 0.3750
Epoch 5/100
20/20 [==============================] - 21s 1s/step - loss: 7.3253 - accuracy: 0.3125
Epoch 6/100
20/20 [==============================] - 21s 1s/step - loss: 4.2617 - accuracy: 0.3750
Epoch 7/100
20/20 [==============================] - 21s 1s/step - loss: 3.8636 - accuracy: 0.4500
Epoch 8/100
20/20 [==============================] - 21s 1s/step - loss: 4.0670 - accuracy: 0.5125
Epoch 9/100
20/20 [==============================] - 20s 1s/step - loss: 3.9811 - accuracy: 0.5250
Epoch 10/100
20/20 [==============================] - 21s 1s/step - loss: 3.6157 - accuracy: 0.5875
Epoch 

In [18]:
MODEL_NAME = "model_saved_100epochs.h5"

model.save('model/' + MODEL_NAME)

In [19]:
from keras.utils import plot_model

plot_model(model, to_file='test.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
